当active的namenode挂掉之后Zookeeper会修改信息。这里介绍一下Zookeeper的“信息修改”是如何做到的。

# 1. Zookeeper文件系统
Zookeeper维护者一个树形的层次结构，我们可以看做一个高可用的文件系统。他没有文件和目录的概念，只有一个节点（node）的概念，称为znode。znode既可以保存数据，也可以作为路径（一个znode同时兼有文件和目录的特点）

![image_1d5uk36h5v5hi9v1lavr615489.png-48.4kB][1]

# 2. Zookeeper命令行客户端操作
在实际应用中我们会将提供Zookeeper服务叫做服务端，将使用Zookeeper提供服务的应用叫做客户端
Zookeeper有一个“命令行客户端”，可以使用
```
$ ZOOKEEPER_HOME/bin/zkCli.sh
```
运行该客户端，可以使用`help`查看具体内容。
![image_1d5ukbjiusce1orped8epjse6m.png-5.9kB][2]

在zookeeper客户端提供了一系列操作，用来操作znode：

- `create [-s] [-e] path data acl`：创建一个znode节点，path为路径，data为数据
- `set path data [version]`：修改节点的值
- `ls path [watch]`：查看指定节点的子节点列表
- `delete path [version]`：删除节点
- `rmr path`：删除节点
- `get path [watch]`：获取节点值
- `quit`：退出client

可以运行一下查看结果：
```
# 1. 查看根目录下目录结构（必须是绝对路径）
[zk: localhost:2181(CONNECTED) 0] ls /
[zookeeper]

# 2. 创建一个znode节点，数据为this-is-data，注意数据不能包含空格，且不识别引号
[zk: localhost:2181(CONNECTED) 1] create /data this-is-data
Created /data

# 3. 现在目录下已经有了我们添加的znode
[zk: localhost:2181(CONNECTED) 2] ls /
[zookeeper, data]

# 4. 查看/data包含数据this-is-data
[zk: localhost:2181(CONNECTED) 3] get /data
this-is-data
cZxid = 0x100000013
ctime = Thu Mar 14 00:15:00 CST 2019
mZxid = 0x100000013
mtime = Thu Mar 14 00:15:00 CST 2019
pZxid = 0x100000013
cversion = 0
dataVersion = 0
aclVersion = 0
ephemeralOwner = 0x0
dataLength = 12
numChildren = 0

# 5. /data下面没有目录
[zk: localhost:2181(CONNECTED) 4] ls /data
[]

# 6. /data下面创建目录
[zk: localhost:2181(CONNECTED) 5] create /data/test this-is-data-test
Created /data/test

# 7. /data出现新建的目录
[zk: localhost:2181(CONNECTED) 6] ls /data
[test]

# 8. /data的数据没有改变
[zk: localhost:2181(CONNECTED) 7] get /data
this-is-data
cZxid = 0x100000013
ctime = Thu Mar 14 00:15:00 CST 2019
mZxid = 0x100000013
mtime = Thu Mar 14 00:15:00 CST 2019
pZxid = 0x100000014
cversion = 1
dataVersion = 0
aclVersion = 0
ephemeralOwner = 0x0
dataLength = 12
numChildren = 1

# 9. /data/test也是有数据的
[zk: localhost:2181(CONNECTED) 8] get /data/test
this-is-data-test
cZxid = 0x100000014
ctime = Thu Mar 14 00:15:39 CST 2019
mZxid = 0x100000014
mtime = Thu Mar 14 00:15:39 CST 2019
pZxid = 0x100000014
cversion = 0
dataVersion = 0
aclVersion = 0
ephemeralOwner = 0x0
dataLength = 17
numChildren = 0
```
# 3. znode的原理

Zookeeper的节点分为两种类型：短暂的和持久的。znode的类型在创建之后就不能再修改了。他们之间的区别在于短暂znode在客户端会话结束后就会被Zookeeper删除掉。

## 3.1 短暂znode

例如：
```
# 启动一个客户端创建一个短暂znode
$ bin/zkCli.sh
[zk: tiny01:2181(CONNECTED) 0] $ create -e /data data
Created /data
[zk: tiny01:2181(CONNECTED) 1] $ ls /
[zookeeper, data]
[zk: tiny01:2181(CONNECTED) 2] $ quit

# 重启客户端
$ bin/zkCli.sh
[zk: tiny01:2181(CONNECTED) 0] ls /
[zookeeper]
```
可以看到短暂的znode在重启客户端后消失了。

当然这个问题，也可以开启两个客户端来模拟。不过这里就不提供方式了。结果就是第二个客户端关闭的时候第一个客户端就看不到第二个客户端创建的znode了。
```
# 第二个客户端关闭的时候，第一个客户端发现不了第二个客户端创建的znode了
[zk: tiny02:2181(CONNECTED) 0] ls /
[zookeeper, test]
[zk: tiny02:2181(CONNECTED) 1] ls /
[zookeeper]
```
<font color="red">注意：短暂znode下面不能创建znode子节点</font>

## 3.2 序号
在创建znode的时候我们也可以添加一个序号，例如：
```
[zk: localhost:2181(CONNECTED) 1] create -s -e /hello helloWorld
Created /hello0000000004
[zk: localhost:2181(CONNECTED) 2] create -s -e /hello helloWorld
Created /hello0000000005
[zk: localhost:2181(CONNECTED) 3] create -s -e /hello helloWorld
Created /hello0000000006
[zk: localhost:2181(CONNECTED) 4] create -s -e /hello helloWorld
Created /hello0000000007
[zk: localhost:2181(CONNECTED) 5] create -s -e /hello helloWorld
Created /hello0000000008
```
他是不断递增的，并且有Zookeeper返回他的znode名字。他的值由父节点来控制。

## 3.3 监听
znode还有一个重要的功能就是监听，当一个znode的状态改变的时候，其他客户端会及时知道znode发生改变了。举个例子，我们有两个客户端：分别在tiny01和tiny02上（注意下面程序的主机名称）：
```
# tiny01上的client创建了一个短暂znode： /test
[zk: tiny01:2181(CONNECTED) 0] create -e /test test
Created /test

# tiny02的client发现了这个znode
[zk: tiny02:2181(CONNECTED) 0] ls /
[zookeeper, test]

# tiny02的client监听这个znode
[zk: tiny02:2181(CONNECTED) 1] ls /test watch
[]

# tiny01退出客户端，所以短暂znode也会被删除
[zk: tiny01:2181(CONNECTED) 1] quit

# tiny02监听到这个znode状态发生了改变，被删除了。
[zk: tiny02:2181(CONNECTED) 2]
WATCHER::

WatchedEvent state:SyncConnected type:NodeDeleted path:/test
```


  [1]: ../imgs/image_1d5uk36h5v5hi9v1lavr615489.png
  [2]: ../imgs/image_1d5ukbjiusce1orped8epjse6m.png